In [1]:
import os, sys
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.feature import get_feat_d_from_ID
from util.genome import is_overlap, NON_K12_EXP_L
pd.options.display.max_columns = 100

In [3]:
all_muts_df = pd.read_pickle("./data/2_3_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(1278, 29)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']}
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085..."
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']}
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']}


In [4]:
TU_objects_df = pd.read_csv("./data/RegulonDB10/tu_objects_tmp.txt",
                            sep="\t",
                            comment='#',
                            header=None,
                            )
TU_objects_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "NUMTU",
    "TU_POSLEFT",
    "TU_POSRIGHT",
    "TU_TYPE",
     "TU_OBJECT_CLASS",
    "TU_OBJECT_ID",
    "TU_OBJECT_NAME",
    "TU_OBJECT_POSLEFT",
    "TU_OBJECT_POSRIGHT",
    "TU_OBJECT_STRAND",
    "TU_OBJECT_COLORCLASS",
    "TU_OBJECT_DESCRIPTION",
    "TU_OBJECT_SIGMA",
    "TU_OBJECT_EVIDENCE",
    "TU_OBJECT_RI_TYPE",
    "TU_OBJECT_TYPE",
    "EVIDENCE"]
TU_objects_df["TU range"] = TU_objects_df.apply(lambda row: (row["TU_POSLEFT"], row["TU_POSRIGHT"]), axis=1)
TU_objects_df.head()

,TRANSCRIPTION_UNIT_ID,NUMTU,TU_POSLEFT,TU_POSRIGHT,TU_TYPE,TU_OBJECT_CLASS,TU_OBJECT_ID,TU_OBJECT_NAME,TU_OBJECT_POSLEFT,TU_OBJECT_POSRIGHT,TU_OBJECT_STRAND,TU_OBJECT_COLORCLASS,TU_OBJECT_DESCRIPTION,TU_OBJECT_SIGMA,TU_OBJECT_EVIDENCE,TU_OBJECT_RI_TYPE,TU_OBJECT_TYPE,EVIDENCE,TU range
0,ECK120008913,3,1825955,1832013,H,PM,ECK120009851,astCp1,1832013,1832013,R,NaN,NaN,Sigma70,Human inference of promoter position,NaN,predicted,NaN,"(1825955, 1832013)"
1,ECK120008913,3,1825955,1832013,H,GN,ECK120003528,astE,1825955,1826923,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
2,ECK120008913,3,1825955,1832013,H,GN,ECK120003529,astB,1826916,1828259,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
3,ECK120008913,3,1825955,1832013,H,GN,ECK120003532,astC,1830762,1831982,R,"0,0,255",nitrogen metabolism,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
4,ECK120008913,3,1825955,1832013,H,GN,ECK120003530,astD,1828256,1829734,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"


In [5]:
reg_TU_mut_hit_df = TU_objects_df.copy()
reg_TU_mut_hit_df = reg_TU_mut_hit_df.drop(
    columns=['NUMTU',
             'TU_TYPE',
             'TU_OBJECT_ID',
             'TU_OBJECT_POSLEFT',
             'TU_OBJECT_POSRIGHT',
             'TU_OBJECT_STRAND',
             'TU_OBJECT_COLORCLASS',
             'TU_OBJECT_DESCRIPTION',
             'TU_OBJECT_SIGMA',
             'TU_OBJECT_EVIDENCE',
             'TU_OBJECT_RI_TYPE',
             'TU_OBJECT_TYPE',
             'EVIDENCE']
)
reg_TU_mut_hit_df = reg_TU_mut_hit_df[reg_TU_mut_hit_df["TU_OBJECT_CLASS"]=="ST"]
reg_TU_mut_hit_df = reg_TU_mut_hit_df.drop_duplicates()
reg_TU_mut_hit_df.head()

,TRANSCRIPTION_UNIT_ID,TU_POSLEFT,TU_POSRIGHT,TU_OBJECT_CLASS,TU_OBJECT_NAME,TU range
12,ECK120008914,1825955,1832327,ST,ArgR,"(1825955, 1832327)"
15,ECK120008914,1825955,1832327,ST,NtrC,"(1825955, 1832327)"
52,ECK120008916,2800586,2804461,ST,NrdR,"(2800586, 2804461)"
53,ECK120008916,2800586,2804461,ST,IscR,"(2800586, 2804461)"
54,ECK120008916,2800586,2804461,ST,Fur,"(2800586, 2804461)"


In [6]:
TU_df = pd.read_csv("./data/RegulonDB10/transcription_unit.txt", sep="\t", comment='#', header=None)
TU_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "PROMOTER_ID",
    "TRANSCRIPTION_UNIT_NAME",
    "OPERON_ID",
    "TRANSCRIPTION_UNIT_NOTE",
    "TU_INTERNAL_COMMENT",
    "KEY_ID_ORG"]


operon_df = pd.read_csv("./data/RegulonDB10/operon.txt", sep="\t", comment='#', header=None)
operon_df.columns = [
    "OPERON_ID",
    "OPERON_NAME",
    "FIRSTGENEPOSLEFT",
    "LASTGENEPOSRIGHT",
    "REGULATIONPOSLEFT",
    "REGULATIONPOSRIGHT",
    "OPERON_STRAND",
    "OPERON_INTERNAL_COMMENT",
    "KEY_ID_ORG"]
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [7]:
def get_TU_ID_set(feat_ID):    
    return set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_ID]["TRANSCRIPTION_UNIT_ID"])


def get_clean_regulon_name_set(reg_str):
    clean_reg_name_set = set()
    if ',' in reg_str:
        clean_reg_name_set |= set(reg_str.split(','))
    else:
        clean_reg_name_set |= set([reg_str])
    return clean_reg_name_set


def get_reg_name_set(TU_ID):
    df = TU_objects_df[TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID]
    regulator_df = df[df["TU_OBJECT_CLASS"]=="ST"]
    reg_name_set = set()
    for reg_str in regulator_df["TU_OBJECT_NAME"]:
        reg_name_set |= get_clean_regulon_name_set(reg_str)
    return reg_name_set


# Must split up regulonDB complex regulatons into single regulators.
assert get_reg_name_set("ECK120009661") == {'NagC', 'CRP', 'Mlc'}  # Contains the complex regulator "NagC,Mlc".


def get_operon_name_set(TU_ID):
    op_name_set = set()
    op_IDs = TU_df[TU_df["TRANSCRIPTION_UNIT_ID"]==TU_ID]["OPERON_ID"].unique()
    for op_ID in op_IDs:
        op_name_set.add(operon_df[operon_df["OPERON_ID"]==op_ID].iloc[0]["OPERON_NAME"])  # Assuming that each operon only has one name
    return op_name_set


def _get_genetic_target_d_from_ID(gen_targ_ID, mut_row):
    gen_targ_d = dict()
    for d in mut_row["genetic features"]:
        if d["RegulonDB ID"] == gen_targ_ID:
            gen_targ_d = d
            break
    return gen_targ_d


def get_regulators(mut_row, reg_TU_mut_hit_df):
    annots_l = []
    if mut_row.exp not in NON_K12_EXP_L:
        for feat_d in mut_row["genomic features"]:
            # !!! This part only handles mutations that land within a genomic feature
            # This used to have the condition '"RegulonDB ID" in feat_d' though I don't feel this is necessary
            # since features should now always have RegulonDB IDs. If they don't, this should crash to indicate the fact.
            if '/' not in feat_d["RegulonDB ID"]:
                TU_hit_set = get_TU_ID_set(feat_d["RegulonDB ID"])  # It's fine to keep the TUs as a set since we don't expect to have a mutated feature affect the same TU twice.

                # This will allow for multiple entries of the same regulon being annotated for a mutation
                # the same Regulon mutated multiple times by a mutation affecting
                # overlapping features or by a mutated feature servicing multiple TUs like in the case of the gcvA gcvB TFBS and the GcvA regulon.            
                for TU in TU_hit_set:
                    for r in get_reg_name_set(TU):
                        annots_l.append({"name": r})

            # This will catch whether genomic features are integenic regions.
            # Get regulon genetic feature link dict through overlapping mutation and regulon ranges.
            # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
            # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
            else:
                reg_TU_mut_hit_df["mutation hit"] = reg_TU_mut_hit_df["TU range"].apply(is_overlap, args=[feat_d["range"]])
                df = reg_TU_mut_hit_df[reg_TU_mut_hit_df["mutation hit"]]
                for _, reg_TU in df.iterrows():            
                    for reg in get_clean_regulon_name_set(reg_TU["TU_OBJECT_NAME"]):
                        annots_l.append({"name": reg})
                reg_TU_mut_hit_df = reg_TU_mut_hit_df.drop(columns=["mutation hit"])
    return annots_l


all_muts_df["regulators"] = all_muts_df.apply(lambda r: get_regulators(r, reg_TU_mut_hit_df), axis=1)
all_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[]
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[]
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[]
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}]
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'NagC'}, {'name': 'CRP'}, {'name': '..."


In [8]:
# Very similar to get_regulators.
def get_regulator_operon_links_d(mut_row, TU_objects_df):
    reg_op_link_d = dict()
    if mut_row.exp not in NON_K12_EXP_L:
        for gen_targ_ID, feat_ID_set in mut_row["genetic feature links"].items():
            for feat_ID in feat_ID_set:
                feat_d = get_feat_d_from_ID(feat_ID, mut_row)
                # This part only handles mutations that land within a genomic feature.
#                 link_exists = False
                
                if '/' not in feat_d["RegulonDB ID"]:
                    operon_name_set = set()
                    reg_name_set = set()
                    TU_hit_set = get_TU_ID_set(feat_d["RegulonDB ID"])
                    for TU in TU_hit_set:
                        reg_name_set |= get_reg_name_set(TU)
                        operon_name_set |= get_operon_name_set(TU)  # using a set because only want one operon per mutated feature TU.
#                     if len(reg_name_set) > 0: link_exists = True
                    for reg_name in reg_name_set:
                        if reg_name not in reg_op_link_d.keys(): reg_op_link_d[reg_name] = []
                        reg_op_link_d[reg_name] += list(operon_name_set)


                # This code is only meant to serve intergenic regions.
                # Get operon genetic feature link dict through overlapping mutated feature and operon ranges.
                # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
                else:
                    TU_objects_df["mutation hit"] = TU_objects_df["TU range"].apply(is_overlap, args=[feat_d["range"]])
                    TU_mut_hit_df = TU_objects_df[TU_objects_df["mutation hit"]]
                    TU_hit_set = set(TU_mut_hit_df["TRANSCRIPTION_UNIT_ID"])
                    operon_name_set = set()
                    reg_name_set = set()
                    for TU in TU_hit_set:
                        reg_name_set |= get_reg_name_set(TU)
                        # using a set because only want one operon per mutated feature TU.
                        operon_name_set |= get_operon_name_set(TU)
#                     if len(reg_name_set) > 0: link_exists = True
                    for reg_name in reg_name_set:
                        if reg_name not in reg_op_link_d.keys(): reg_op_link_d[reg_name] = []
                        reg_op_link_d[reg_name] += list(operon_name_set)
                    # cleanup
                    if "mutation hit" in TU_objects_df.columns.values:
                        TU_objects_df = TU_objects_df.drop(columns=["mutation hit"])
                
                # Per mutation feature
                # Need to be looking at previously populated level of annotations depdending on how the current level is being populated
                # for regulon, it just means that if there are no operons, go immediately to genomic features.
#                 if link_exists == False:
#                     if "unknown" not in reg_op_link_d.keys(): reg_op_link_d["unknown"] = []
#                     if len(operon_name_set) > 0: # need to always check the nearest annotation type for defined features
#                         reg_op_link_d["unknown"] += list(operon_name_set)
#                     else:
#                         reg_op_link_d["unknown"].append(feat_d["RegulonDB ID"])

        return reg_op_link_d


all_muts_df["regulator links"] = all_muts_df.apply(lambda r: get_regulator_operon_links_d(r, TU_objects_df), axis=1)
all_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{}
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{}
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{}
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'NagC'}, {'name': 'CRP'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'NagC': ['nagBAC-ump..."


In [9]:
all_muts_df.to_pickle("./data/2_4_df.pkl")